<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/Pandas_Regression_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
from google.colab import files
uploaded = files.upload()

Saving weatherHistory.csv to weatherHistory.csv


In [0]:
weather_hist = pd.read_csv('weatherHistory.csv')

In [0]:
weather_hist['Wind Speed (km/h)'].max()

63.8526

Найдём день, когдабыл самый сильный ветер:

In [0]:
weather_hist[weather_hist['Wind Speed (km/h)'] == 63.8526]['Formatted Date']

12181    2007-01-29 13:00:00.000 +0100
Name: Formatted Date, dtype: object

In [0]:
weather_hist['Temperature (C)'].min()

-21.822222222222226

А теперь выясним, какой же день был самым холодным:

In [0]:
weather_hist[weather_hist['Temperature (C)'] == -21.822222222222226]['Formatted Date']

54847    2012-02-10 07:00:00.000 +0100
Name: Formatted Date, dtype: object

In [0]:
weather_hist['Formatted Date']=pd.to_datetime(weather_hist['Formatted Date'], utc=True)

In [0]:
weather_hist['year']=weather_hist['Formatted Date'].dt.year

In [0]:
weather_hist['month']=weather_hist['Formatted Date'].dt.month

In [0]:
weather_hist['week']=weather_hist['Formatted Date'].dt.week

In [0]:
weather_hist['day']=weather_hist['Formatted Date'].dt.day

In [0]:
weather_hist['mon']=weather_hist['Formatted Date'].dt.to_period('M')

/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [0]:
weather_hist['wee']=weather_hist['Formatted Date'].dt.to_period('W')

/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [0]:
set(weather_hist[weather_hist['Summary'] == 'Rain']['year'])

{2016}

Исходя из этих данных — дождь за всё время наблюдения был только в 2016 году. Проверим для кратковременных дождей:

In [0]:
set(weather_hist[weather_hist['Summary'] == 'Light Rain']['year'])

{2012, 2016}

Значит, дождя в 2014 году не было

Сгруппируем самую большую скорость ветра по месяцам и отсортируем от самого ветренного — к наименее ветренному:

In [0]:
columns_to_show = ['Wind Speed (km/h)']
weather_hist.groupby('mon')[columns_to_show].sum().sort_values(by='Wind Speed (km/h)', ascending=False)

,Wind Speed (km/h)
mon,
2009-03,12560.4794
2008-03,10879.1242
2010-03,10870.0277
2013-03,10766.3759
2006-03,10729.5874
...,...
2015-08,5519.7562
2006-07,5352.2679
2011-11,4947.5783


Теперь найдём самый влажный год:

In [0]:
columns_to_show = ['Humidity']
weather_hist.groupby('year')[columns_to_show].sum().sort_values(by='Humidity', ascending=False)

,Humidity
year,
2010,6999.63
2006,6721.98
2016,6682.70
2013,6606.77
2014,6555.24
2011,6447.43
2015,6415.42
2009,6195.48
2008,6159.68


Теперь найдём неделю с самой переменчивой погодой:

In [0]:
columns_to_show = ['Temperature (C)']
weather_hist.groupby('wee')[columns_to_show].agg(['min', 'max'])

Temperature (C)          
                                  min       max
wee                                            
2005-12-26/2006-01-01        0.577778  7.438889
2006-01-02/2006-01-08       -4.933333  7.505556
2006-01-09/2006-01-15       -6.205556  2.927778
2006-01-16/2006-01-22      -11.533333  6.200000
2006-01-23/2006-01-29      -14.088889  5.166667
...                               ...       ...
2016-11-28/2016-12-04        0.038889  6.327778
2016-12-05/2016-12-11       -0.716667  5.255556
2016-12-12/2016-12-18       -1.355556  4.333333
2016-12-19/2016-12-25       -1.855556  3.588889
2016-12-26/2017-01-01       -2.177778  3.016667

[575 rows x 2 columns]